In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from numpy import mean
from numpy import std
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# first running random forest regression on the aggregate dataset

In [7]:
# load the dataset as pandas DataFrame  
# read pandas from xlsx file

df = pd.read_excel('/Users/andrei-macpro/Documents/Data/classification/accoustics/extraction_3.0/cg_meal_prosody_regression.xlsx')

In [8]:
# from df remove columns "intensity_percentile_1_99_range"  
df = df.drop(columns=['intensity_percentile_1_99_range', 'loudness_slidingwindow_percentile_1_99_range', 
                      'intensity_percentile_1_99_range', 'rms_mean', 'rms_std','rms_percentile_1_99_range'])
df

,s_id,loudness,loudness_slidingwindow_mean,loudness_slidingwindow_std,shannon_entropy,log_energy,intensity_mean,intensity_std,num_zerocrossings,zerocrossing_rate,...,ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer,DAI,Rinab
0,1043,-34.446908,-35.403694,1.375606,142.739830,-34.580744,0.000663,0.000945,2430.389610,0.055111,...,0.019818,0.055707,0.186161,1.609937,0.089611,0.127614,0.210271,0.268832,0,5
1,1047,-32.398648,-33.175423,1.211462,232.410584,-32.274857,0.001090,0.001482,2239.384409,0.050780,...,0.020974,0.056635,0.196950,1.680808,0.093018,0.130937,0.192693,0.279055,6,15
2,1053,-37.768684,-38.192081,1.054975,80.255764,-37.342031,0.000288,0.000368,2018.995169,0.045782,...,0.019615,0.055019,0.189859,1.644138,0.092376,0.127882,0.212157,0.277128,2,3
3,1059,-39.755333,-40.307227,1.103294,53.033801,-39.332350,0.000177,0.000241,2115.352531,0.047967,...,0.030004,0.083881,0.206689,1.745600,0.103354,0.151471,0.265969,0.310063,0,31
4,1062,-30.313932,-31.115852,1.184623,335.555989,-30.107822,0.001810,0.002546,2434.317507,0.055200,...,0.020881,0.055995,0.184451,1.599252,0.085962,0.122827,0.203812,0.257887,0,3
5,1069,-27.197428,-27.967966,1.165350,498.194548,-27.235098,0.002683,0.003694,2559.767773,0.058045,...,0.020708,0.055506,0.186587,1.618162,0.085064,0.123049,0.202719,0.255192,6,13
6,1073,-35.021337,-35.606290,1.257240,148.970011,-34.870191,0.000811,0.001391,2351.367347,0.053319,...,0.019031,0.049424,0.200114,1.757795,0.091684,0.132469,0.226441,0.275051,0,4
7,1079,-33.568416,-34.183199,0.993793,165.774269,-33.612551,0.000692,0.000884,2674.395480,0.060644,...,0.017500,0.047138,0.180660,1.582862,0.085097,0.121770,0.198181,0.255292,1,9
8,1080,-34.987514,-35.885206,1.275373,150.322138,-34.606154,0.000657,0.000789,2397.278014,0.054360,...,0.020204,0.057035,0.189430,1.641084,0.092665,0.128741,0.206998,0.277995,0,5
9,1082,-29.734226,-30.440179,1.136631,327.131265,-29.705032,0.001501,0.001709,2339.430034,0.053048,...,0.018027,0.049111,0.177641,1.567841,0.082110,0.118797,0.196294,0.246330,2,5


In [15]:
# turn the dataset except the first column and last 2 columns into a numpy array 
X = df.iloc[:, 1:-2].values
s_id = df.iloc[:,0].values
dai = df.iloc[:,-2].values
rinab = df.iloc[:,-1].values

array([0, 6, 2, 0, 0, 6, 0, 1, 0, 2, 1, 1, 0, 2, 2, 1, 2, 0, 0, 4, 1, 0,
       2, 4, 4, 3, 0, 4, 0, 8, 0, 0, 0, 1, 1, 1, 0, 1, 1, 6, 5, 2, 5, 1,
       3, 2, 0, 0, 4, 1, 1, 1, 1, 3, 2, 5])

In [10]:
# run a random forest regression model on the dataset 
# define the model
model = RandomForestRegressor()
# define the evaluation procedure
cv = KFold(n_splits=5)
# evaluate the model
scores = cross_val_score(model, df.iloc[:, 1:], df.iloc[:, 0], scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
# report performance
print('Mean MSE: %.3f (%.3f)' % (mean(scores), std(scores)))


Mean MSE: -55741.973 (79812.141)


In [ ]:
# read pandas from xlsx file
